# Predicting Credit Card Fraud using Scikit-Learn and Synthetic Training & Test Data

Completed for my Codecademy Data Scientist (Machine Learning Specialist) certification course.

Robert Hall

10/10/2024.
